In [2]:
!rm -rf sample_data
!git clone https://github.com/elsheikh21/boats-classification

Cloning into 'boats-classification'...
remote: Enumerating objects: 6788, done.
remote: Counting objects: 100% (6788/6788), done.
remote: Compressing objects: 100% (6780/6780), done.
remote: Total 6788 (delta 5), reused 6784 (delta 4), pack-reused 0
Receiving objects: 100% (6788/6788), 333.85 MiB | 35.34 MiB/s, done.
Resolving deltas: 100% (5/5), done.
Checking out files: 100% (6690/6690), done.


In [1]:
import os
import pandas as pd 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.callbacks import CSVLogger
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2

Using TensorFlow backend.


### Step 1: set our directory variables

- **training_dir**: where the training data will be found in set of subfolders
<br>
- **testing_dir**: where the testing data will be found
<br>
- **ground_truth_path**: file to compare against our model predicition


In [0]:
training_dir = os.path.join(os.getcwd(), 'boats-classification', 'training_set')
testing_dir = os.path.join(os.getcwd(), 'boats-classification', 'testing_set', 'test')

ground_truth_dir = os.path.join(testing_dir, 'ground_truth.txt')

### Step2: Data preprocessing & augmentation

- to make sure that the model doesn't encounter the same image twice, this helps avoid overfitting, as well as, generalizing better
- **train_datagen**: Is responsible for the images augmentation
- **train_generator** : applies the augmentations from the data gen to the training set and saves it in the path <code>/data/training/_processed</code>

In [0]:
# The augmentation for training set
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True
)

# The augmentation for testing set
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

### Step3: Prepare the data
- Generate batches of image data (and their labels) directly from training folders
- **batch_size**: refers to the number of training examples utilised in one iteration, for CPU powered we use {32, 64}, for GPU boosted we use {128, 256}.
- Save the processed images to view them for analysis
- **target_size**: images generated of width and height (128, 128)
- **Class mode**: Categorical as we are trying to classify categories of different boats
- Use those generators to train our models
- Always save the model, compile once, reuse for as long as we wish


In [4]:
batch_size = 128

# The generator will read pictures found in
# subfolers of training_data & generate batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        training_dir, batch_size=batch_size, class_mode='categorical'
)
train_samples = train_generator.samples
train_num_classes = train_generator.num_classes

Found 4717 images belonging to 19 classes.


In [5]:
# Further insights about our testing data-set
df = pd.read_csv(ground_truth_dir, delimiter=';')
df.columns = ['filename', 'class']

to_remove_list = ['Cacciapesca', 'Caorlina',
                  'Lanciamaggioredi10mMarrone', 'Sanpierota', 'VigilidelFuoco',
                  'SnapshotBarcaParziale', 'SnapshotBarcaMultipla',
                  'Mototopocorto']

new_df = df[~df['class'].isin(to_remove_list)]

# Similar generator, for testing data
test_generator = test_datagen.flow_from_dataframe(new_df, testing_dir)

testing_samples = len(new_df.values)
unique_testing_classes = new_df.groupby('class')['class'].nunique()
testing_classes_num = len(unique_testing_classes.values)

Found 1671 images belonging to 19 classes.


In [6]:
# Visualize the data
print('Training set: ({}, {})'.format(train_samples, train_num_classes))
print('Testing set: ({}, {})'.format(
    testing_samples, testing_classes_num))

Training set: (4717, 19)
Testing set: (1671, 19)


### Step4: Training Our model
- Other than fighting the overfitting issue using data augmentation, we try **model entropic capacity**, so modulating it depends on the number of layers and the size of each layer.

- **Dropout** also helps reduce overfitting, by preventing a layer from seeing twice the exact same pattern, thus acting in a way analoguous to data augmentation

- Models used 
  - LeNet model
    - first set of CONV => RELU => POOL
    - second set of CONV => RELU => POOL
    - set of FC => RELU layers
    - softmax classifier
  - AlexNet model
 
- Compile the model
- Fit generated data to the model
- Save model weights and model itself

In [0]:
def lenet_model(img_shape, train_classes,
                weights_path=None, visualize_summary=False):
    model = Sequential()

    # first set of CONV => RELU => POOL
    model.add(Conv2D(20, (5, 5), input_shape=img_shape))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # second set of CONV => RELU => POOL
    model.add(Conv2D(50, (5, 5)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # set of FC => RELU layers
    model.add(Flatten())
    model.add(Dense(500))
    model.add(Activation("relu"))

    # softmax classifier
    model.add(Dense(train_classes))
    model.add(Activation("softmax"))

    # if weightsPath is specified load the weights
    if weights_path is not None:
        model.load_weights(weights_path)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])

    if(visualize_summary):
        print('\nLeNet Model:')
        print(model.summary())

    return model


In [0]:
def fit_save_lenet_model(lenet_model, train_generator, save_model=True):
    # Tracking our model
    csv_logger1 = CSVLogger('lenet_model_training.log')
    # Fit generated data to the model
    print('\nFitting the model process has begun...\n')
    lenet_model.fit_generator(
        train_generator,
        steps_per_epoch=(train_generator.n//train_generator.batch_size),
        epochs=50,
        verbose=1,
        callbacks=[csv_logger1],
        use_multiprocessing=True,
        workers=0
    )
    print('\nLeNet Model fitting process has ended successfully.')
    print("Log file is generated with the name 'lenet_model_training.log'")

    if(save_model):
        # Save model weights
        lenet_model.save_weights('lenet_model_weights.h5')
        # Save model for further analysis
        lenet_model.save("lenet_model.h5")

    # for better visualizing our model log
    lenet_log_df = pd.read_csv('lenet_model_training.log', delimiter=',')
    lenet_log_df.to_csv('lenet_model_training.csv')
    print("Log file is generated with the name 'lenet_model_training.csv'")

In [0]:
from numpy import savetxt

def predict_eval_lenet_model(lenet_model, test_generator):
    print('LeNet Model evaluation & prediction process starting...')
    try:
        predict = lenet_model.predict_generator(
            test_generator, test_generator.n // test_generator.batch_size,
            verbose=1)
        scores = lenet_model.evaluate_generator(
            test_generator, test_generator.n // test_generator.batch_size,
            verbose=1)
        savetxt('LeNet_scores.txt', scores)
        savetxt('LeNet_predictions.txt', predict)
        print('Evaluation and prediction scores are saved.')
    except BaseException as error:
        print('An exception occurred: {}'.format(error))

In [0]:
def alexnet_model(img_shape, n_classes, l2_reg,
                  weights_path=None, visualize_summary=False):

    # Initialize model
    alexnet = Sequential()

    # Layer 1
    alexnet.add(Conv2D(96, (11, 11), input_shape=img_shape,
                       padding='same', kernel_regularizer=l2(l2_reg)))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(MaxPooling2D(pool_size=(2, 2)))

    # Layer 2
    alexnet.add(Conv2D(256, (5, 5), padding='same'))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(MaxPooling2D(pool_size=(2, 2)))

    # Layer 3
    alexnet.add(ZeroPadding2D((1, 1)))
    alexnet.add(Conv2D(512, (3, 3), padding='same'))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(MaxPooling2D(pool_size=(2, 2)))

    # Layer 4
    alexnet.add(ZeroPadding2D((1, 1)))
    alexnet.add(Conv2D(1024, (3, 3), padding='same'))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))

    # Layer 5
    alexnet.add(ZeroPadding2D((1, 1)))
    alexnet.add(Conv2D(1024, (3, 3), padding='same'))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(MaxPooling2D(pool_size=(2, 2)))

    # Layer 6
    alexnet.add(Flatten())
    alexnet.add(Dense(3072))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(Dropout(0.5))

    # Layer 7
    alexnet.add(Dense(4096))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(Dropout(0.5))

    # Layer 8
    alexnet.add(Dense(n_classes))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('softmax'))

    if weights_path is not None:
        alexnet.load_weights(weights_path)

    alexnet.compile(loss='categorical_crossentropy',
                    optimizer='adam', metrics=['accuracy'])

    if(visualize_summary):
        print('\nAlexNet Model:')
        print(alexnet.summary())

    return alexnet

In [0]:

def fit_save_alexnet_model(alexnet_model, train_generator, save_model=True):
    # Tracking the models
    csv_logger2 = CSVLogger('alexnet_model_training.log')

    print('\nFitting the model process has begun...\n')
    alexnet_model.fit_generator(
        train_generator,
        steps_per_epoch=(train_generator.n//train_generator.batch_size),
        epochs=50,
        verbose=1,
        callbacks=[csv_logger2],
        use_multiprocessing=True,
        workers=0
    )
    print('\nAlexNet Model fitting process has ended successfully.')
    print("Log file is generated with the name 'alexnet_model_training.log'")

    if(save_model):
        # Save model weights
        alexnet_model.save_weights('alexnet_model_weights.h5')
        # Save model for further analysis
        alexnet_model.save("alexnet_model.h5")

    alexnet_log_df = pd.read_csv('alexnet_model_training.log', delimiter=',')
    alexnet_log_df.to_csv('alexnet_model_training.csv')
    print("Log file is generated with the name 'alexnet_model_training.csv'")



In [0]:

def predict_eval_alexnet_model(alexnet_model, test_generator):
    print('\nAlexNet Model evaluation & prediction process starting...')
    try:
        predict = alexnet_model.predict_generator(
            test_generator, test_generator.n // test_generator.batch_size,
            verbose=1)
        scores = alexnet_model.evaluate_generator(
            test_generator, test_generator.n // test_generator.batch_size,
            verbose=1)
        savetxt('AlexNet_scores.txt', scores)
        savetxt('AlexNet_predictions.txt', predict)
        print('Evaluation and prediction scores are saved.')
    except BaseException as error:
        print('An exception occurred: {}'.format(error))

In [0]:
from keras.optimizers import SGD

def vgg_16(img_size, weights_path=None, visualize_summary=True):
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=img_size))
    model.add(Conv2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(
        optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    if(visualize_summary):
        print('\nVGG16 Model:')
        print(model.summary())

    return model


def fit_save_vgg_16_model(vgg16_model, train_generator, save_model=True):
    # Tracking our model
    csv_logger = CSVLogger('VGG16_model_training.log')
    # Fit generated data to the model
    print('\nFitting the model process has begun...\n')
    vgg16_model.fit_generator(
        train_generator,
        steps_per_epoch=(train_generator.n//train_generator.batch_size),
        epochs=50,
        verbose=1,
        callbacks=[csv_logger],
        use_multiprocessing=True,
        workers=0
    )

    if(save_model):
        vgg16_model.save('VGG16_model.h5')
        vgg16_model.save_weights('VGG16_model_weights.h5')
    
    # for better visualizing our model log
    vgg16_log_df = pd.read_csv('VGG16_model_training.log', delimiter=',')
    vgg16_log_df.to_csv('VGG16_model_training.csv')
    print("Log file is generated with the name 'VGG16_model_training.csv'")


def predict_eval_vgg_16_model(vgg16_model, test_generator):
    print('VGG16 Model evaluation & prediction process starting...')
    try:
        predict = lenet_model.predict_generator(
            test_generator, test_generator.n // test_generator.batch_size,
            verbose=1)
        scores = lenet_model.evaluate_generator(
            test_generator, test_generator.n // test_generator.batch_size,
            verbose=1)
        savetxt('VGG16_scores.txt', scores)
        savetxt('VGG16_predictions.txt', predict)
        print('Evaluation and prediction scores are saved.')
    except BaseException as error:
        print('An exception occurred: {}'.format(error))


In [13]:
# LeNet Model
lenet_model = lenet_model(img_shape=(256, 256, 3),
                          train_classes=train_num_classes,
                          weights_path=None, visualize_summary=True)

fit_save_lenet_model(lenet_model, train_generator, save_model=True)


LeNet Model:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 252, 252, 20)      1520      
_________________________________________________________________
activation_1 (Activation)    (None, 252, 252, 20)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 126, 20)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 122, 122, 50)      25050     
_________________________________________________________________
activation_2 (Activation)    (None, 122, 122, 50)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 61, 61, 50)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 186050)            0      

In [0]:
# AlexNet Model
alexnet_model = alexnet_model(img_shape=(256, 256, 3),
                              n_classes=train_num_classes, l2_reg=0.0,
                              weights_path=None, visualize_summary=True)

fit_save_alexnet_model(alexnet_model, train_generator, save_model=True)

In [27]:
vgg16_model = vgg_16(img_size=(256, 256, 3),
                          weights_path=None, visualize_summary=True)

fit_save_vgg_16_model(lenet_model, train_generator, save_model=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), a


VGG16 Model:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_17 (ZeroPaddi (None, 258, 258, 3)       0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 256, 256, 64)      1792      
_________________________________________________________________
zero_padding2d_18 (ZeroPaddi (None, 258, 258, 64)      0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 128, 128, 64)      0         
_________________________________________________________________
zero_padding2d_19 (ZeroPaddi (None, 130, 130, 64)      0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 128, 128, 128)     73856  

### Step5: Test our models (LeNet, AlexNet, VGG16)

In [23]:
test_lenet_model = predict_eval_lenet_model(lenet_model, test_generator)

LeNet Model evaluation & prediction process starting...
52/52 [==============================] - 9s 172ms/step
Evaluation and prediction scores are saved.


In [0]:
test_alexnet_model = predict_eval_alexnet_model(test_generator)

In [29]:
test_vgg16_model = predict_eval_vgg_16_model(vgg16_model, test_generator)

VGG16 Model evaluation & prediction process starting...
52/52 [==============================] - 9s 173ms/step
Evaluation and prediction scores are saved.
